# TMDB API로 데이터 수집

## TMDB 드라마 데이터 전수 수집

1. 수집 대상: TMDB TV Series & Seasons 데이터
2. 수집 기간: 2016년 1월 1일 ~ 2025년 11월 30일 (약 11년치)
3. 수집 범위: TV Series과 Seasons 전체 페이지 (에피소드 제외)
4. 수집 방식: 100% 비동기(Pure Async) 방식
5. 수집 컬럼: 시리즈 54개, 시즌 13개
6. 수집 조건: 페이지 누락 0%, TMDB API 초당 요청 수(40 req/s) 제한 대응, 500페이지 제한 대응
7. 추가 사항: 시리즈는 providers를 3종류(flatrate/rent/buy) & 전체 국가로 다시 수집 + imdb_id 수집, 시즌은 vote_count 삭제
8. 수집 결과: 총 244,961개(Series 100403개, Seasons 144558개), 약 117분(약 2시간) 소요

In [4]:
# ==========================================================
# TMDB TV SERIES + SEASONS COLLECTOR (FINAL ULTIMATE V9)
# - Series 54 cols / Seasons 13 cols
# - append_to_response 1회 호출 최적화
# - providers(flatrate/rent/buy) 세미콜론 문자열
# - 시즌 기반 결측치 보완
# - nest_asyncio 지원 (Jupyter 호환)
# - 상세 진행률 표시 (속도/남은시간)
# - discover 500페이지 제한 대응 (날짜 분할 + 단일일 500p 시 10k까지만 수집)
# - ✅ temp는 JSONL로 저장 → CSV 파싱 에러 방지
# ==========================================================

import os
import asyncio
import aiohttp
import async_timeout
from collections import deque
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import json
from dotenv import load_dotenv

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except Exception:
    pass

# ==========================================================
# 환경 변수
# ==========================================================
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")
if not API_KEY:
    raise ValueError("❌ TMDB_API_KEY missing in .env")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}
TIMEOUT = 10
MAX_RETRIES = 5
MAX_CALLS_PER_SECOND = 36  # TMDB 40req/s 여유

START_DATE = "2016-01-01"
END_DATE   = "2025-11-30"

# Temp(JSONL)
SERIES_TEMP_JSONL  = Path("tv_series_temp.jsonl")
SEASONS_TEMP_JSONL = Path("tv_seasons_temp.jsonl")

# Output
SERIES_CSV   = "tv_series_FULL.csv"
SEASONS_CSV  = "tv_seasons_FULL.csv"
SERIES_PARQ  = "tv_series_FULL.parquet"
SEASONS_PARQ = "tv_seasons_FULL.parquet"

# ==========================================================
# Rate Limiter
# ==========================================================
class AsyncRateLimiter:
    def __init__(self, max_calls=36, period=1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
        self.lock = asyncio.Lock()

    async def acquire(self):
        async with self.lock:
            now = asyncio.get_event_loop().time()

            # 기간 지난 호출 제거
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()

            # 초당 호출 수 초과 시 대기
            if len(self.calls) >= self.max_calls:
                wait_for = self.period - (now - self.calls[0]) + 0.01
                await asyncio.sleep(max(0, wait_for))

                now = asyncio.get_event_loop().time()
                while self.calls and now - self.calls[0] > self.period:
                    self.calls.popleft()

            self.calls.append(now)

rate_limiter = AsyncRateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# 공통 helper
# ==========================================================
def safe_json(obj):
    """CSV에 넣기 위한 안전한 JSON 문자열 (줄바꿈 제거)"""
    if not obj:
        return ""
    return json.dumps(obj, ensure_ascii=False).replace("\n", " ").replace("\r", " ")

def safe_text(s):
    """긴 텍스트(overview 등) 줄바꿈 제거"""
    if s is None:
        return ""
    return str(s).replace("\n", " ").replace("\r", " ")

def list_to_str(lst, key="name"):
    if not lst:
        return ""
    return ", ".join(str(x.get(key)) for x in lst if x.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(x.get(key)) for x in lst if x.get(key))

def append_jsonl(path: Path, records):
    """dict 리스트를 JSONL 파일에 append"""
    if not records:
        return
    with path.open("a", encoding="utf-8") as f:
        for rec in records:
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

def load_jsonl_to_df(path: Path) -> pd.DataFrame:
    """JSONL → DataFrame (깨진 라인 있으면 스킵)"""
    if not path.exists():
        return pd.DataFrame()
    rows = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rows.append(json.loads(line))
            except json.JSONDecodeError:
                # 일부 라인 깨져도 전체는 계속
                continue
    return pd.DataFrame(rows)

# ==========================================================
# TMDB GET
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        return None

    params = params or {}
    params["api_key"] = API_KEY
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"

    await rate_limiter.acquire()

    try:
        async with async_timeout.timeout(TIMEOUT):
            async with session.get(url, params=params, headers=HEADERS) as resp:

                # 레이트 리밋
                if resp.status == 429:
                    wait = float(resp.headers.get("Retry-After", 2)) * (2 ** retry)
                    await asyncio.sleep(wait)
                    return await tmdb_get(session, path, params, retry + 1)

                # Not Found
                if resp.status == 404:
                    return None

                # 서버 에러 (5xx)
                if 500 <= resp.status < 600:
                    await asyncio.sleep(2 ** retry)
                    return await tmdb_get(session, path, params, retry + 1)

                resp.raise_for_status()
                return await resp.json()

    except Exception:
        await asyncio.sleep(2 ** retry)
        return await tmdb_get(session, path, params, retry + 1)

# ==========================================================
# DISCOVER
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)


async def collect_ids_in_range(session, start, end, depth=0):
    """
    TV Series ID 수집 (500 페이지 제한 회피)
    - 날짜 범위가 500 페이지 넘으면 자동 분할
    - start == end (하루 기준)이며 여전히 500+ 페이지면 → 10,000개까지만 수집 가능 (TMDB 한계)
    """
    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    indent = "  " * depth
    print(f"{indent}📅 Discover {start} ~ {end} → pages={total_pages}, results={total_results}")

    if total_results == 0:
        return set()

    # 단일일 + 500페이지 초과 → 500페이지만 전수 수집 가능 (TMDB API 구조적 한계)
    if start == end and total_pages > 500:
        print(f"{indent}⚠️ {start}: {total_pages}페이지 > 500 → 최대 10,000개까지만 수집 (나머지 누락)")
        ids = {x["id"] for x in res1}
        tasks = [fetch_discover_page(session, p, start, end)
                 for p in range(2, 501)]  # 최대 500 페이지
        for coro in asyncio.as_completed(tasks):
            r, _, _ = await coro
            ids |= {x["id"] for x in r}
        print(f"{indent}➡ {start} 수집 ID 수: {len(ids):,} (상한 10,000)")
        return ids

    # 500 페이지 이하 → 그대로 전수 수집
    if total_pages <= 500:
        ids = {x["id"] for x in res1}
        if total_pages > 1:
            tasks = [
                fetch_discover_page(session, p, start, end)
                for p in range(2, total_pages + 1)
            ]
            for coro in asyncio.as_completed(tasks):
                r, _, _ = await coro
                ids |= {x["id"] for x in r}
        print(f"{indent}✅ 수집 완료: {len(ids):,} IDs")
        return ids

    # 그 외: 날짜 범위 분할
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt   = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    print(f"{indent}✂️ 500페이지 초과 → {start}~{mid} / {right}~{end} 로 분할")

    left_ids  = await collect_ids_in_range(session, start, mid, depth + 1)
    right_ids = await collect_ids_in_range(session, right, end, depth + 1)
    merged = left_ids | right_ids
    print(f"{indent}🔁 병합 완료: {len(merged):,} IDs")
    return merged

# ==========================================================
# 시즌 수집 (13개 컬럼)
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    """
    시즌 상세 정보 수집 (13개 컬럼)

    ⚠️ TMDB Season API는 networks 필드를 직접 제공하지 않음
    → 시리즈 networks를 fallback으로 사용
    """

    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    episodes = (data or {}).get("episodes") or []
    runtimes = [
        ep.get("runtime")
        for ep in episodes
        if isinstance(ep.get("runtime"), (int, float))
    ]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else 0  # None 대신 0으로 통일

    return {
        "_id": f"{sid}_{sn}",
        "season_id": meta.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": sn,
        "name": meta.get("name"),
        "air_date": meta.get("air_date"),
        "overview": safe_text(meta.get("overview")),
        "vote_average": meta.get("vote_average"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": meta.get("episode_count"),
        "avg_episode_runtime": avg_rt,
    }

# ==========================================================
# 시리즈 + 시즌 (append_to_response 1회, Series 54 컬럼)
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    # 🔥 한 번에 모든 부가정보 로드
    params = {
        "append_to_response": (
            "external_ids,keywords,reviews,aggregate_credits,watch/providers"
        )
    }
    data = await tmdb_get(session, f"/tv/{sid}", params=params)
    if not data:
        return None, []

    # imdb_id
    imdb_id = (data.get("external_ids") or {}).get("imdb_id")

    # providers (전세계 / 3분류) → 세미콜론 문자열
    providers_data = (data.get("watch/providers") or {}).get("results", {}) or {}

    flat = {}
    rent = {}
    buy  = {}

    for cc, info in providers_data.items():
        if not isinstance(info, dict):
            continue

        flatrate_list = [
            p.get("provider_name")
            for p in (info.get("flatrate") or [])
            if p.get("provider_name")
        ]
        rent_list = [
            p.get("provider_name")
            for p in (info.get("rent") or [])
            if p.get("provider_name")
        ]
        buy_list = [
            p.get("provider_name")
            for p in (info.get("buy") or [])
            if p.get("provider_name")
        ]

        if flatrate_list:
            flat[cc] = flatrate_list
        if rent_list:
            rent[cc] = rent_list
        if buy_list:
            buy[cc] = buy_list

    def fmt(d):
        if not d:
            return ""
        return "; ".join(f"{cc}: {', '.join(v)}" for cc, v in d.items())

    providers_flat = fmt(flat)
    providers_rent = fmt(rent)
    providers_buy  = fmt(buy)

    genres   = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep  = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids   = list_ids_to_str(networks)

    # ===== SERIES 54개 =====
    series = {
        "id": data.get("id"),
        "type": "tv_series",
        "imdb_id": imdb_id,
        "title": data.get("name"),
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": (
            ", ".join(map(str, data.get("episode_run_time", [])))
            if data.get("episode_run_time") else ""
        ),
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(x.get("id")) for x in genres if x.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) if data.get("languages") else "",
        "last_air_date": data.get("last_air_date"),

        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": safe_text(last_ep.get("overview")),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),

        # ✅ JSON 문자열 1개 컬럼 (A방식)
        "next_episode_to_air": safe_json(data.get("next_episode_to_air")),

        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),
        "origin_country": ", ".join(data.get("origin_country", [])) if data.get("origin_country") else "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": safe_text(data.get("overview")),
        "popularity": data.get("popularity"),
        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),
        "seasons": "; ".join(
            f"S{s.get('season_number')}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or [])
            if s.get("season_number") is not None
        ),

        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),

        "review": "",
        "keyword": "",
        "top_cast": "",
        "directors": "",
        "writers": "",

        "providers_flatrate": providers_flat,
        "providers_rent": providers_rent,
        "providers_buy": providers_buy,
        "poster_path": data.get("poster_path"),
    }

    # 리뷰 (최대 5개) — 텍스트로 유지
    reviews = (data.get("reviews") or {}).get("results", [])
    rv = []
    for r in reviews[:5]:
        auth = r.get("author", "")
        rating = (r.get("author_details") or {}).get("rating")
        rt = f"({rating})" if rating is not None else ""
        cont = safe_text(r.get("content"))[:200]
        rv.append(f"{auth}{rt}: {cont}")
    series["review"] = " || ".join(rv)

    # 키워드 (이름만 콤마조인)
    kw = (data.get("keywords") or {}).get("results", [])
    series["keyword"] = ", ".join([k["name"] for k in kw if k.get("name")])

    # cast/crew (aggregate_credits)
    credits = data.get("aggregate_credits") or {}
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c["name"] for c in cast[:4] if c.get("name")])

    dirs = set()
    wrs = set()
    for c in crew:
        nm = c.get("name")
        if not nm:
            continue
        for j in (c.get("jobs") or []):
            job = j.get("job")
            if job == "Director":
                dirs.add(nm)
            if job in ["Writer", "Screenplay", "Story"]:
                wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs))
    series["writers"]   = ", ".join(sorted(wrs))

    # 시즌 수집
    seasons_meta = data.get("seasons") or []
    season_tasks = [
        fetch_single_season(session, sid, data.get("name", ""), net_names, net_ids, s)
        for s in seasons_meta
        if s.get("season_number") is not None
    ]

    season_records = []
    for coro in asyncio.as_completed(season_tasks):
        x = await coro
        if x:
            season_records.append(x)

    return series, season_records

# ==========================================================
# 시즌 기반 결측치 보완
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완 시작")

    # episode_run_time (문자열 형태 → 시즌 평균 runtime으로 채움)
    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = (
        series_df["episode_run_time"].isna()
        | series_df["episode_run_time"].astype(str).str.strip().isin(
            ["", "nan", "none", "[]", "{}"]
        )
    )
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    # number_of_episodes (시즌 total_episodes 합)
    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = (
        series_df.loc[mask2, "id"].map(epmap)
    )

    # number_of_seasons (시즌 개수)
    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = (
        series_df.loc[mask3, "id"].map(seasmap)
    )

    # last_air_date (시즌 air_date 기준 마지막 날짜)
    lastair = (
        seasons_df.sort_values("air_date")
        .groupby("series_id")["air_date"]
        .last()
    )
    mask4 = (
        series_df["last_air_date"].isna()
        | series_df["last_air_date"].astype(str).str.strip().isin(
            ["", "nan", "nat", "none"]
        )
    )
    series_df.loc[mask4, "last_air_date"] = (
        series_df.loc[mask4, "id"].map(lastair)
    )

    print("✅ 시즌 기반 결측치 보완 완료\n")
    return series_df

# ==========================================================
# MAIN
# ==========================================================
async def main():
    print("=" * 100)
    print("🚀 TMDB FULL SERIES + SEASONS COLLECTOR (FINAL ULTIMATE V9)")
    print("=" * 100)
    print(f"📅 기간: {START_DATE} ~ {END_DATE}")
    print("📊 Series 54 cols / Seasons 13 cols")
    print("🔥 append_to_response 통합 + providers 세미콜론 문자열")
    print("✨ nest_asyncio 지원 + 상세 진행률 + discover 500p 대응")
    print("💾 TEMP: JSONL 사용 → CSV 파서 에러 방지")
    print("=" * 100)

    t0 = datetime.now()

    # temp 파일 삭제
    for f in [SERIES_TEMP_JSONL, SEASONS_TEMP_JSONL]:
        if f.exists():
            f.unlink()

    async with aiohttp.ClientSession() as session:

        print("\n📌 [1단계] Discover → TV Series ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        total_ids = len(ids)
        print(f"✨ 총 시리즈 수(추정): {total_ids:,}개")

        print("\n📌 [2단계] TV 상세 + 시즌 수집")
        batch_size = 400
        sem = asyncio.Semaphore(25)

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        batch_start_time = datetime.now()

        for start_idx in range(0, total_ids, batch_size):
            part = ids[start_idx:start_idx + batch_size]
            tasks = [fetch_one(x) for x in part]

            batch_series = []
            batch_seasons = []

            for coro in asyncio.as_completed(tasks):
                s, ss = await coro
                if s:
                    batch_series.append(s)
                if ss:
                    batch_seasons.extend(ss)

                processed += 1

                # 🔥 상세 진행률 표시 (100개마다)
                if processed % 100 == 0 or processed == total_ids:
                    elapsed = (datetime.now() - batch_start_time).total_seconds() / 60
                    rate = processed / elapsed if elapsed > 0 else 0
                    remain = (total_ids - processed) / rate if rate > 0 else 0
                    pct = processed / total_ids * 100
                    print(
                        f"⏱ 진행: {processed:,}/{total_ids:,} ({pct:5.1f}%) | "
                        f"속도: {rate:.1f}/분 | 남음: {remain:.1f}분"
                    )

            # 배치 저장 (JSONL)
            append_jsonl(SERIES_TEMP_JSONL, batch_series)
            append_jsonl(SEASONS_TEMP_JSONL, batch_seasons)

    print("\n📌 [3단계] DataFrame 로드 + 스키마 정리 (JSONL → DF)")

    SERIES_COLS = [
        "id","type","imdb_id","title","adult","backdrop_path","created_by",
        "episode_run_time","first_air_date","genres","genre_ids","homepage",
        "in_production","languages","last_air_date","last_episode_to_air_id",
        "last_episode_to_air_name","last_episode_to_air_overview",
        "last_episode_to_air_vote_average","last_episode_to_air_vote_count",
        "last_episode_to_air_air_date","last_episode_to_air_episode_number",
        "last_episode_to_air_production_code","last_episode_to_air_runtime",
        "last_episode_to_air_season_number","last_episode_to_air_show_id",
        "last_episode_to_air_still_path","next_episode_to_air",
        "networks","number_of_episodes","number_of_seasons","origin_country",
        "original_language","original_name","overview","popularity",
        "production_companies","production_countries","seasons",
        "spoken_languages","status","tagline","type_detail","vote_average",
        "vote_count","review","keyword","top_cast","directors","writers",
        "providers_flatrate","providers_rent","providers_buy","poster_path",
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name",
        "air_date","overview","vote_average","network_names","network_ids",
        "total_episodes","avg_episode_runtime",
    ]

    df_series  = load_jsonl_to_df(SERIES_TEMP_JSONL)
    df_seasons = load_jsonl_to_df(SEASONS_TEMP_JSONL)

    # 컬럼 순서/누락 정리
    df_series  = df_series.reindex(columns=SERIES_COLS)
    df_seasons = df_seasons.reindex(columns=SEASON_COLS)

    # 중복 제거
    df_series  = df_series.drop_duplicates("id")
    df_seasons = df_seasons.drop_duplicates(["series_id", "season_number"])

    print(f"➡ Series DF:  {df_series.shape}")
    print(f"➡ Seasons DF: {df_seasons.shape}")

    print("\n📌 [4단계] 시즌 기반 결측치 보완")
    df_series = fill_series_gaps(df_series, df_seasons)

    print("\n📌 [5단계] CSV / Parquet 저장")

    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
        print("✅ Parquet 저장 완료")
    except Exception as e:
        print(f"⚠️ Parquet 저장 중 오류 (무시): {e}")

    # temp 삭제 (원하면 남겨도 됨)
    for f in [SERIES_TEMP_JSONL, SEASONS_TEMP_JSONL]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds() / 60

    print("\n" + "=" * 100)
    print("🎉 TMDB TV SERIES + SEASONS FULL 수집 완료 (FINAL ULTIMATE V9)")
    print("=" * 100)
    print(f"📌 Series 수: {len(df_series):,}개")
    print(f"📌 Season 수: {len(df_seasons):,}개")
    print(f"📌 Series 컬럼 수: {len(SERIES_COLS)} (목표: 54)")
    print(f"📌 Season 컬럼 수: {len(SEASON_COLS)} (목표: 13)")
    print(f"⏱ 총 소요시간: {elapsed:.1f}분 ({elapsed/60:.2f}시간)")
    print("=" * 100)

# ==========================================================
# RUN
# ==========================================================
if __name__ == "__main__":
    asyncio.run(main())


✅ nest_asyncio 적용 완료
🚀 TMDB FULL SERIES + SEASONS COLLECTOR (FINAL ULTIMATE V9)
📅 기간: 2016-01-01 ~ 2025-11-30
📊 Series 54 cols / Seasons 13 cols
🔥 append_to_response 통합 + providers 세미콜론 문자열
✨ nest_asyncio 지원 + 상세 진행률 + discover 500p 대응
💾 TEMP: JSONL 사용 → CSV 파서 에러 방지

📌 [1단계] Discover → TV Series ID 수집
📅 Discover 2016-01-01 ~ 2025-11-30 → pages=5021, results=100403
✂️ 500페이지 초과 → 2016-01-01~2020-12-15 / 2020-12-16~2025-11-30 로 분할
  📅 Discover 2016-01-01 ~ 2020-12-15 → pages=1967, results=39333
  ✂️ 500페이지 초과 → 2016-01-01~2018-06-24 / 2018-06-25~2020-12-15 로 분할
    📅 Discover 2016-01-01 ~ 2018-06-24 → pages=864, results=17268
    ✂️ 500페이지 초과 → 2016-01-01~2017-03-28 / 2017-03-29~2018-06-24 로 분할
      📅 Discover 2016-01-01 ~ 2017-03-28 → pages=414, results=8276
      ✅ 수집 완료: 8,276 IDs
      📅 Discover 2017-03-29 ~ 2018-06-24 → pages=450, results=8992
      ✅ 수집 완료: 8,992 IDs
    🔁 병합 완료: 17,268 IDs
    📅 Discover 2018-06-25 ~ 2020-12-15 → pages=1104, results=22065
    ✂️ 500페이지 초과 → 201

## 수집 결과 점검

In [ ]:
import pandas as pd

SERIES_CSV = "tv_series_2016_2025_FULL(100,403개).csv"
SEASONS_CSV = "tv_seasons_2016_2025_FULL(144,558개).csv"

# ======================================================
# 1. Load
# ======================================================
print("📌 Loading CSV...")
df_series = pd.read_csv(SERIES_CSV, low_memory=False)
df_seasons = pd.read_csv(SEASONS_CSV, low_memory=False)

print(f"➡ Series:  {df_series.shape}")
print(f"➡ Seasons: {df_seasons.shape}")
print("=" * 80)

# ======================================================
# 2. Series 컬럼 검증
# ======================================================
series_cols_expected = [
    "id","type","imdb_id","title","adult","backdrop_path","created_by",
    "episode_run_time","first_air_date","genres","genre_ids","homepage",
    "in_production","languages","last_air_date","last_episode_to_air_id",
    "last_episode_to_air_name","last_episode_to_air_overview",
    "last_episode_to_air_vote_average","last_episode_to_air_vote_count",
    "last_episode_to_air_air_date","last_episode_to_air_episode_number",
    "last_episode_to_air_production_code","last_episode_to_air_runtime",
    "last_episode_to_air_season_number","last_episode_to_air_show_id",
    "last_episode_to_air_still_path","next_episode_to_air",
    "networks","number_of_episodes","number_of_seasons","origin_country",
    "original_language","original_name","overview","popularity",
    "production_companies","production_countries","seasons",
    "spoken_languages","status","tagline","type_detail","vote_average",
    "vote_count","review","keyword","top_cast","directors","writers",
    "providers_flatrate","providers_rent","providers_buy","poster_path",
]

print("📌 [1] 시리즈 컬럼 검증")
missing = [c for c in series_cols_expected if c not in df_series.columns]
extra = [c for c in df_series.columns if c not in series_cols_expected]

print("총 컬럼 수:", len(df_series.columns), "(기대: 54)")
print("누락된 컬럼:", missing)
print("추가된 컬럼:", extra)
print("=" * 80)

# ======================================================
# 3. Season 컬럼 검증
# ======================================================
season_cols_expected = [
    "_id","season_id","series_id","series_name","season_number","name",
    "air_date","overview","vote_average","network_names","network_ids",
    "total_episodes","avg_episode_runtime",
]

print("📌 [2] 시즌 컬럼 검증")
missing_s = [c for c in season_cols_expected if c not in df_seasons.columns]
extra_s = [c for c in df_seasons.columns if c not in season_cols_expected]

print("총 시즌 컬럼 수:", len(df_seasons.columns), "(기대: 13)")
print("누락된 컬럼:", missing_s)
print("추가된 컬럼:", extra_s)
print("=" * 80)

# ======================================================
# 4. 결측치 비율
# ======================================================
print("📌 [3] Series 결측치 비율 Top 20")
print(df_series.isna().mean().sort_values(ascending=False))
print("=" * 80)

print("📌 [4] Seasons 결측치 비율")
print(df_seasons.isna().mean().sort_values(ascending=False))
print("=" * 80)

# ======================================================
# 5. ID 중복 체크
# ======================================================
print("📌 [5] ID 중복 체크")
print("Series 중복 id:", df_series["id"].duplicated().sum())

pair = df_seasons.duplicated(["series_id", "season_number"]).sum()
print("Seasons 중복 (series_id, season_number):", pair)
print("=" * 80)

# ======================================================
# 6. 날짜 파싱 테스트
# ======================================================
def date_parse_fail(df, col):
    fails = 0
    for x in df[col].dropna():
        try:
            pd.to_datetime(x, errors="raise")
        except Exception:
            fails += 1
    return fails

print("📌 [6] 날짜 필드 파싱 실패 수")
print("first_air_date:", date_parse_fail(df_series, "first_air_date"))
print("last_air_date:", date_parse_fail(df_series, "last_air_date"))
print("last_episode_to_air_air_date:", date_parse_fail(df_series, "last_episode_to_air_air_date"))
print("season.air_date:", date_parse_fail(df_seasons, "air_date"))
print("=" * 80)

# ======================================================
# 7. 기본 통계
# ======================================================
print("📌 [7] 시리즈 숫자형 통계")
print(df_series[["popularity", "vote_average", "vote_count",
                 "number_of_episodes", "number_of_seasons"]].describe())
print("=" * 80)

print("📌 [8] 시즌 숫자형 통계")
print(df_seasons[["vote_average","total_episodes",
                  "avg_episode_runtime"]].describe())
print("=" * 80)

print("🎉 데이터 점검 완료")


📌 Loading CSV...
➡ Series:  (100403, 54)
➡ Seasons: (144558, 13)
📌 [1] 시리즈 컬럼 검증
총 컬럼 수: 54 (기대: 54)
누락된 컬럼: []
추가된 컬럼: []
📌 [2] 시즌 컬럼 검증
총 시즌 컬럼 수: 13 (기대: 13)
누락된 컬럼: []
추가된 컬럼: []
📌 [3] Series 결측치 비율 Top 20
last_episode_to_air_production_code    0.996175
next_episode_to_air                    0.988078
review                                 0.981544
providers_rent                         0.967053
tagline                                0.949474
providers_buy                          0.895810
writers                                0.804837
created_by                             0.781112
keyword                                0.693784
last_episode_to_air_still_path         0.691453
directors                              0.689232
last_episode_to_air_overview           0.678416
providers_flatrate                     0.657540
imdb_id                                0.602163
homepage                               0.595112
production_companies                   0.562065
last_episode_to_air_ru

In [28]:
# 크롤링 조건
cr = df_series[(df_series['vote_count'] >= 30) & (df_series['imdb_id'].notna())]
cr

,id,type,imdb_id,title,adult,backdrop_path,created_by,episode_run_time,first_air_date,genres,...,vote_count,review,keyword,top_cast,directors,writers,providers_flatrate,providers_rent,providers_buy,poster_path
4,62127,tv_series,tt3322310,Marvel's Iron Fist,False,/xHCfWGlxwbtMeeOnTvxUCZRGnkk.jpg,Scott Buck,54,2017-03-17,"Action & Adventure, Drama, Sci-Fi & Fantasy, C...",...,2598,"magatsu(1.0): UGH, this show is awful. Simply ...","martial arts, kung fu, crime fighter, superher...","Finn Jones, Jessica Henwick, Jessica Stroup, T...","Andy Goddard, David Dobkin, Deborah Chow, Farr...","Cristine Chambers, Daniel Shattuck, Declan de ...",AD: Disney Plus; AL: Disney Plus; AR: Disney P...,NaN,"AT: Amazon Video, Google Play Movies, maxdome ...",/4l6KD9HhtD6nCDEfg10Lp6C6zah.jpg
10,18712,tv_series,tt12364156,The Search,False,/gdDzcLwFo7hCZperXRoHAPeSYm1.jpg,Santiago Limón,42,2020-06-12,"Crime, Drama, Comedy",...,346,NaN,"journalism, investigation, politics, satire, m...","Regina Blandón, Dario Yazbek Bernal, Diana Bov...","Catalina Aguilar Mastretta, Katina Medina Mora...","Fabián Archondo, Santiago Limón, Silvia Jiménez",AD: Netflix; AE: Netflix; AG: Netflix; AL: Net...,NaN,NaN,/kHfbT8U8W9exQWYW49h3aVFdReA.jpg
14,45871,tv_series,tt10375568,Locked Up: The Oasis,False,/7wm4CH6LbsFul4wWmmfXGcStVsk.jpg,Iván Escobar,50,2020-04-20,Drama,...,2002,NaN,spin off,"Maggie Civantos, Najwa Nimri, Itziar Castro, D...","Miguel Ángel Vivas, Sandra Gallego","Iván Escobar, J. M. Ruiz Córdoba, Lucía Carballal",JP: Hulu,NaN,NaN,/7EOWkNGxXq2MBZxfV3ZxjRM4vlH.jpg
15,39852,tv_series,tt6048596,The Sinner,False,/lVWku9LX2qdJTtAbRzaCDhhWeTK.jpg,Derek Simonds,45,2017-08-02,"Crime, Drama",...,1164,"Gustavo SANTIAGO(8.0): A very solid story, tha...","based on novel or book, anthology, murder, mem...","Bill Pullman, Jessica Hecht, Dohn Norwood, Mat...","Adam Bernstein, Andrew McCarthy, Antonio Campo...","Ahmadu Garba, Bradford Winters, Derek Simonds,...",AD: Netflix; AE: Netflix; AG: Netflix; AL: Net...,"DK: SF Anytime; FI: SF Anytime, Telia Play; NO...","AT: Amazon Video, Google Play Movies, maxdome ...",/rmibFGdqOe0kKKhPls0jVOdZCWw.jpg
17,47663,tv_series,tt17371078,The Ark,False,/nUEcobWsTpvOh23US0MRZXIU8jt.jpg,Dean Devlin,52,2023-02-01,"Sci-Fi & Fantasy, Mystery",...,146,MovieGuys(4.0): The Ark is one of those B grad...,"space colony, space travel, speculative, medit...","Christie Burke, Reece Ritchie, Richard Fleeshm...","Dean Devlin, Hannah Espia, Jonathan Glassner, ...","Dean Devlin, India Sage Wilson, James DeLorean...","AT: Amazon Prime Video, Amazon Prime Video wit...",NaN,"US: Amazon Video, Apple TV, Google Play Movies...",/rec774m02XNLSjU9qm5z6UUfMRl.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93440,294726,tv_series,tt37439438,Amy Bradley Is Missing,False,/bx26v6trU13B2VfzSMfpMJe7OGb.jpg,NaN,49,2025-07-16,"Documentary, Crime, Mystery",...,35,NaN,"miniseries, true crime, missing person","Iva Bradley, Ron Bradley, Brad Bradley, Erin S...","Ari Mark, Phil Lott","Ari Mark, Phil Lott",AD: Netflix; AE: Netflix; AG: Netflix; AL: Net...,NaN,NaN,/bS7hWmAGEFXbgSYyKqX0uapxWO7.jpg
95117,297464,tv_series,tt37065878,The Guest,False,/wJBYXpzKHZNWnkfMuXhr40daPan.jpg,NaN,42,2025-09-24,Drama,...,31,NaN,psychological thriller,"Laura Londoño, Carmen Villalobos, Jason Day, K...","Israel Sanchez, Klych Lopez","Dario Venegas, Lina Uribe",AD: Netflix; AE: Netflix; AG: Netflix; AL: Net...,NaN,NaN,/fBMRxziwviMBVLOCe1i4qtwRXKj.jpg
95332,298234,tv_series,tt37898320,aka Charlie Sheen,False,/qSxabAdLMiJWIJUb7hbSVvwMm4s.jpg,Andrew Renzi,92,2025-09-10,Documentary,...,34,NaN,"rise and fall, miniseries, redemption, recover...","Charlie Sheen, Jon Cryer, Sean Penn, Ramon Est...",Andrew Renzi,NaN,AD: Netflix; AE: Netflix; AG: Netflix; AL: Net...,NaN,NaN,/pHAM8Fzek7xwTbqmaxPXAjCAjz1.jpg
96042,300258,tv_series,tt36595665,Los hilos del pasado,False,/AgOJ5Jgtj1ZVbBfWD4dArXbUdaC.jpg,José Alberto Castro,42,2025-09-10,"Soap, Drama",...,31,NaN,NaN,"Bá

In [30]:
# 크롤링 제외 데이터 1
cr_x_1 = df_series[(df_series['vote_count'] >= 30) & (df_series['imdb_id'].isna())]
cr_x_1

,id,type,imdb_id,title,adult,backdrop_path,created_by,episode_run_time,first_air_date,genres,...,vote_count,review,keyword,top_cast,directors,writers,providers_flatrate,providers_rent,providers_buy,poster_path
2336,69746,tv_series,NaN,La Fan,False,/fmy7OLj4ObNctWS54KrEdkUZOLu.jpg,NaN,0,2017-01-17,Comedy,...,31,NaN,NaN,"Angélica Vale, Juan Pablo Espinosa, Scarlet Or...",NaN,NaN,"US: Peacock Premium, NBC, Peacock Premium Plus",NaN,NaN,/k8bHG4wsOKCFjwJZoUN68EC3BOy.jpg
4240,74321,tv_series,NaN,Will & Grace,False,/aqQQaeMQZF4nobKYi3W1QshRfGE.jpg,"David Kohan, Max Mutchnick",22,2017-09-28,Comedy,...,88,NaN,"new york city, sequel, lgbt, reboot, sitcom, g...","Eric McCormack, Debra Messing, Sean Hayes, Meg...",James Burrows,"Adam Barr, Alex Herschlag, Gus Hickey, John Qu...",AU: Stan; GB: Now TV; IE: Now TV; US: Hulu,NaN,"AT: Amazon Video, Google Play Movies, maxdome ...",/iRsgY1TRNTccltEGk9V9wNqqKOx.jpg
6283,79063,tv_series,NaN,Cunk on...,False,/4nXIoy86Z5kUgKvWNZYyLEqtp7p.jpg,Charlie Brooker,30,2018-04-03,Comedy,...,130,NaN,"england, mockumentary, spoof, historical, dead...","Diane Morgan, Ashley Jackson, Ruth Adams, Jim ...","Christian Watt, Lorry Powles","Ben Caudell, Charlie Brooker, Eli Goldstone, J...",NaN,NaN,NaN,/2ZEZF2IVQkaAlfC8l3Oh7j8ZjDy.jpg
6796,80079,tv_series,NaN,American Idol,False,/37ElQkysfiWKa04GbcB2hgzVznm.jpg,NaN,97,2018-03-11,Reality,...,31,"tmdb51616167(9.0): American idol, a season and...","competition, judge, talent, music competition,...","Ryan Seacrest, Lionel Richie, Katy Perry, Luke...",NaN,NaN,AE: TOD; BH: TOD; DZ: TOD; EG: TOD; IQ: TOD; J...,NaN,NaN,/nF1LPz7dPo6ivAqQjuLa4xGGN78.jpg
9288,86197,tv_series,NaN,El Barón,False,/5V6hsQXCPG5qqt6tojf0Ln4K29O.jpg,NaN,45,2019-01-30,Drama,...,44,NaN,NaN,"Francisco Angelini, María Elisa Camargo, Jorge...",NaN,NaN,"US: Peacock Premium, NBC, Peacock Premium Plus",NaN,NaN,/c9MwIFMZupT4ziJ69pEkuKxrRaU.jpg
9690,87012,tv_series,NaN,The Great British Bake Off,False,/psrUEfxIP7PwBk1vFQ9h0NSzIau.jpg,Anna Beattie,60,2017-08-29,Reality,...,40,NaN,"food, baking, reality competition","Paul Hollywood, Prue Leith, Noel Fielding, Mat...",NaN,NaN,NaN,NaN,NaN,/bG51CdS25evMAdilV4oYkQx5sGn.jpg
12096,91759,tv_series,NaN,Come Home Love: Lo and Behold,False,/dyFTt1a9ZpFdKE96kPlE9fQvXOJ.jpg,NaN,22,2017-02-06,"Family, Comedy, Drama",...,44,NaN,"family, sitcom, ethical, modern drama, heartwa...","Danny Lau Dan, Angela Tong Ying-Ying, Koni Lui...",Luo Zhen Yue,"Anna Lee, Ma Jun Ying, Yuan Bao Hui",NaN,NaN,NaN,/lgD4j9gUGmMckZpWWRJjorWqGVT.jpg
12943,93544,tv_series,NaN,Top Boy,False,/jAz9AZpznu3ELhUp1XiMEeQv2Se.jpg,Ronan Bennett,54,2019-09-13,"Crime, Drama, Action & Adventure",...,145,NaN,"london, england, loyalty, drug dealer, despair...","Kane Robinson, Ashley Walters, Micheal Ward, A...","Aneil Karia, Brady Hood, Koby Adom, Myriam Raj...",NaN,AD: Netflix; AE: Netflix; AG: Netflix; AL: Net...,NaN,NaN,/mGZpOEaLZTRzWeQMq5SZM5BbDZg.jpg
16159,100001,tv_series,NaN,"‘Pedazo de acordeón’, un viaje a través de la ...",False,/duCxXu14gaC17JU5wFTZjmWfFeo.jpg,NaN,0,2019-06-24,NaN,...,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/1IyJnBviBi3xyCiTYlsaVKDtcYG.jpg
16484,100617,tv_series,NaN,Transformers: War for Cybertron: Siege,False,/bxZrG6wsmKMoUC1q4ByTWDRHaOh.jpg,NaN,24,2020-07-30,"Animation, Action & Adventure, Kids, Sci-Fi & ...",...,293,NaN,"mecha, based on toy, giant robot, battle, anim...","Brook Chalmers, Jason Marnocha, Frank Todaro, ...",Takashi Kamei,"Brandon Easton, F.J. DeSanto, Gavin Hignight, ...",AD: Netflix; AE: Netflix; AG: Netflix; AL: Net...,NaN,NaN,/lBUPQLk9gEg8Mg6noHcew8Aag0M.jpg


In [31]:
# 크롤링 제외 데이터 2
cr_x_2 = df_series[(df_series['vote_count'] < 30) & (df_series['imdb_id'].isna())]
cr_x_2

,id,type,imdb_id,title,adult,backdrop_path,created_by,episode_run_time,first_air_date,genres,...,vote_count,review,keyword,top_cast,directors,writers,providers_flatrate,providers_rent,providers_buy,poster_path
3,13247,tv_series,NaN,Sylvanian Families,False,NaN,NaN,3,2018-11-01,"Animation, Kids",...,1,NaN,NaN,NaN,NaN,NaN,"AT: Amazon Prime Video, RTL+, Amazon Prime Vid...",NaN,NaN,/yz3GLV1siEvGJvEyQ5URW6lxjMC.jpg
11,32438,tv_series,NaN,Love Me Again,False,/5twj9qQvg9EpghzsNQ7krLE6tUW.jpg,NaN,47,2023-03-23,"Drama, Comedy",...,0,NaN,"romance, girl group, music industry, pop band,...","Suppasit Jongcheveevat, Lapassalan Jiravechsoo...","Khets Thunthup, Petch Warayu Rakkul",Chanchana Homsap,NaN,NaN,NaN,/rhLUyH6Vz0DFFeGIsjzCjqRCVGb.jpg
13,19976,tv_series,NaN,Deal or No Deal,False,/rbI3slWbuBqPaqDkFMt7SYRdw95.jpg,John de Mol,0,2023-03-06,Reality,...,2,NaN,game show,Katlego Maboe,NaN,NaN,NaN,NaN,NaN,/tyFvCih3yQWqQrXgxzRTU67cypX.jpg
16,43043,tv_series,NaN,Journey,False,/dxU3u7UOPAHWdUASHKriV6rFrKj.jpg,Cheran,43,2024-01-12,"Family, Drama",...,1,NaN,NaN,"Aari Arjuna, Divyabharathi, Prasanna, Kalaiyar...",Cheran,Cheran,US: Pure Flix,NaN,GB: Amazon Video; US: Amazon Video,/p93749EeZizD2GNjiK2OSld2Ygz.jpg
19,40353,tv_series,NaN,Born to Shine,False,/chiwN8Kpzaw7WGqo0vNLxIe3L5j.jpg,NaN,60,2024-03-12,"Drama, Family",...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/5gWtCsojgQkwvMSK60NYIqmeMTF.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100397,306641,tv_series,NaN,Devote 100 Days to Loving You,False,NaN,NaN,0,2025-11-25,Drama,...,0,NaN,"romance, fantasy","Gao Jicai, Gao Weiman, Chou Ziyi, Song Weiyang",NaN,NaN,NaN,NaN,NaN,/xkprRrHWzi3fhLKBoKC3LqLUldx.jpg
100398,306823,tv_series,NaN,在恋爱开始之前,False,/7OAAtBUL4TL0VLDdYBSB8CfQOsd.jpg,NaN,0,2025-11-25,Reality,...,0,NaN,NaN,"Eliza Liang, Luo Yizhou, Christy Chung Lai-Tai...",NaN,NaN,NaN,NaN,NaN,/kzLYvsyP40ZqGrSb9yPm8APV941.jpg
100399,306719,tv_series,NaN,Hvem passer på grisene?,False,/x5Ew4m04TIdKLKRorhe2CCGqaaU.jpg,NaN,40,2024-12-12,Documentary,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/eSQYVhpyPJZTVesCWFiB54u9h7V.jpg
100401,307156,tv_series,NaN,มิติสยอง เชื่อเป็น...ไม่เชื่อตาย,False,NaN,NaN,0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# 크롤링 제외 데이터 3
cr_x_3 = df_series[(df_series['vote_count'] < 30) & (df_series['imdb_id'].notna())]
cr_x_3

,id,type,imdb_id,title,adult,backdrop_path,created_by,episode_run_time,first_air_date,genres,...,vote_count,review,keyword,top_cast,directors,writers,providers_flatrate,providers_rent,providers_buy,poster_path
0,46458,tv_series,tt0272406,Private Lives,False,/9yDxLFmWr5MDfwsWo9uwfbRM3d4.jpg,Tracy Borman,0,2019-03-25,Documentary,...,0,NaN,historical figure,NaN,Bruce Burgess,NaN,NaN,NaN,NaN,/toGbVYxgR4m9qJBpqNs5xMKZkJT.jpg
1,1680,tv_series,tt31638181,Best of The Beat with Ari Melber,False,/3yaiscBrWNDH6a5SPmc72gK4S3J.jpg,NaN,60,2024-01-13,NaN,...,0,NaN,NaN,Ari Melber,NaN,NaN,NaN,NaN,NaN,/jV5Gkv94xwr67oSDYHQN8kUXCTN.jpg
2,45734,tv_series,tt8381430,Street Smart,False,NaN,NaN,30,2018-08-05,News,...,0,NaN,NaN,"Tahir Bilgiç, Rob Shehadie, Neel Kolhatkar, Ma...",NaN,NaN,NaN,NaN,NaN,/54ROJhhs8nHLaKhXxrxMSFNCj9u.jpg
5,57971,tv_series,tt11710778,Think Big,False,NaN,NaN,0,2020-02-07,Comedy,...,1,NaN,NaN,"Hanna Plaß, Anselm Bresgott, Milena Dreißig, V...",NaN,NaN,NaN,NaN,NaN,/qP8XAHj0gBuyydwVgsHirWCg4Ym.jpg
6,44945,tv_series,tt2231004,Audio-Files,False,NaN,NaN,30,2024-04-10,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100382,306880,tv_series,tt9389662,Cam and Leon,False,NaN,NaN,0,2018-06-01,"Kids, Comedy, Animation",...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/i46BEVVKpLSVduUYoHRBkBkgEHW.jpg
100384,306480,tv_series,tt16160870,Popcorn,False,/80I9HYelCwWDWCH5PicQWtmvZyr.jpg,Pierre-Alexis Bizot,136,2019-09-14,"Talk, Comedy, News",...,0,NaN,"talk show, youtube, popcorn, twitch streamer, ...","Pierre-Alexis Bizot, Marie Charlotte Palot, Au...",NaN,NaN,NaN,NaN,NaN,/kc9pIt2XJP7FOVpvv5lqGHjwW5Z.jpg
100388,306944,tv_series,tt35488501,A Sámi Wedding,False,NaN,Åse Kathrin Vuolab,0,2025-09-08,"Comedy, Drama",...,0,NaN,"sibling rivalry, wedding, loveless marriage, i...","Sara Margrethe Oskal, Ánte Siri, Inga Marja Ut...","Pål Jackman, Åse Kathrin Vuolab","Jörgen Hjerdt, Pauline Wolff, Åse Kathrin Vuolab",NaN,NaN,NaN,NaN
100395,306886,tv_series,tt38454779,Firottiye,False,/wCZj3GABz8Ypwc5DWTFrgHzNp1o.jpg,Jagjeet Singh Dhanoa,0,2025-11-28,Action & Adventure,...,0,NaN,NaN,"Sukh Dandiwal, Rai Singh, Rehmat Ghuman",Jagjeet Singh Dhanoa,NaN,NaN,NaN,NaN,/mbgkVypbw7RaOWdYA8TSTjycpHB.jpg


In [ ]:
# 시리즈 데이터 총 개수 확인 (이상 없음)
total = len(cr) + len(cr_x_1) + len(cr_x_2) + len(cr_x_3)
total

100403